# CS4001/4042 Assignment 1, Part B, Q2
In Question B1, we used the Category Embedding model. This creates a feedforward neural network in which the categorical features get learnable embeddings. In this question, we will make use of a library called Pytorch-WideDeep. This library makes it easy to work with multimodal deep-learning problems combining images, text, and tables. We will just be utilizing the deeptabular component of this library through the TabMlp network:

In [32]:
!pip install pytorch-widedeep

You should consider upgrading via the '/Users/advait/Desktop/gitpositories/SC4001-Assignment/env/bin/python -m pip install --upgrade pip' command.


In [33]:
SEED = 42

import os

import random
random.seed(SEED)

import numpy as np
np.random.seed(SEED)

import pandas as pd

from pytorch_widedeep.preprocessing import TabPreprocessor
from pytorch_widedeep.models import TabMlp, WideDeep
from pytorch_widedeep import Trainer
from pytorch_widedeep.metrics import R2Score

>Divide the dataset (‘hdb_price_prediction.csv’) into train and test sets by using entries from the year 2020 and before as training data, and entries from 2021 and after as the test data.

In [34]:
df = pd.read_csv('hdb_price_prediction.csv')

# TODO: Enter your code here
train_data = df[df['year'] <= 2020]
test_data = df[df['year'] >= 2021]

>Refer to the documentation of Pytorch-WideDeep and perform the following tasks:
https://pytorch-widedeep.readthedocs.io/en/latest/index.html
* Use [**TabPreprocessor**](https://pytorch-widedeep.readthedocs.io/en/latest/examples/01_preprocessors_and_utils.html#2-tabpreprocessor) to create the deeptabular component using the continuous
features and the categorical features. Use this component to transform the training dataset.
* Create the [**TabMlp**](https://pytorch-widedeep.readthedocs.io/en/latest/pytorch-widedeep/model_components.html#pytorch_widedeep.models.tabular.mlp.tab_mlp.TabMlp) model with 2 linear layers in the MLP, with 200 and 100 neurons respectively.
* Create a [**Trainer**](https://pytorch-widedeep.readthedocs.io/en/latest/pytorch-widedeep/trainer.html#pytorch_widedeep.training.Trainer) for the training of the created TabMlp model with the root mean squared error (RMSE) cost function. Train the model for 100 epochs using this trainer, keeping a batch size of 64. (Note: set the *num_workers* parameter to 0.)

In [35]:
# TODO: Enter your code here

# Define the target variable and the names of the continuous and categorical variables
target = ['resale_price']
continuous_cols = ['dist_to_nearest_stn', 'dist_to_dhoby', 'degree_centrality', 'eigenvector_centrality', 'remaining_lease_years', 'floor_area_sqm']
categorical_cols = ['month', 'town', 'flat_model_type', 'storey_range']

preprocessor = TabPreprocessor(embed_cols=categorical_cols, continuous_cols=continuous_cols)
x_train = preprocessor.fit_transform(train_data)
y_train = train_data[target].values

model = TabMlp(
    mlp_hidden_dims=[200, 100],
    column_idx=preprocessor.column_idx,
    cat_embed_input=preprocessor.cat_embed_input,
    continuous_cols=continuous_cols
)

# Combine the TabMlp model with any other models you want to use
wide = WideDeep(deeptabular=model)

# Set up Trainer and train
trainer = Trainer(
    wide, 
    objective="root_mean_squared_error", 
    metrics=[R2Score], 
    num_workers=0
)

# Fit the model

trainer.fit(X_tab=x_train, target=y_train, n_epochs=100, batch_size=64)



/Users/advait/Desktop/gitpositories/SC4001-Assignment/env/lib/python3.9/site-packages/pytorch_widedeep/preprocessing/tab_preprocessor.py:334: UserWarning: Continuous columns will not be normalised
  warnings.warn("Continuous columns will not be normalised")
epoch 100: 100%|██████████| 1366/1366 [00:14<00:00, 92.86it/s, loss=4.97e+4, metrics={'r2': 0.8878}] 


>Report the test RMSE and the test R2 value that you obtained.

In [37]:
# TODO: Enter your code here

import math
from sklearn.metrics import r2_score, mean_squared_error

x_test = preprocessor.transform(test_data)
y_test = test_data[target].values

predictions = trainer.predict(X_tab=x_test,batch_size=64)

print("RMSE: ", math.sqrt(mean_squared_error(test_data['resale_price'], predictions)))
print("R2: ", r2_score(test_data['resale_price'], predictions))


predict: 100%|██████████| 1128/1128 [00:03<00:00, 368.52it/s]


RMSE:  96905.11983419929
R2:  0.6719107686934478
